In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [2]:
lr = 1e-3
batch_size = 64
epochs = 5

In [3]:
loss_fn = nn.CrossEntropyLoss()

In [4]:
optimizer = torch.optim.SGD(model.parameters(),lr = lr)

In [11]:
def train_loop(dataloader,model,loss_fn,optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch,(X,y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred,y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch%100==0:
            loss,current = loss.item(),(batch+1)*len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            #每隔100个batch跟踪一次训练进度

def test_loop(dataloader,model,loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss,correct = 0,0

    with torch.no_grad():
        for X,y in dataloader:
            pred = model(X)
            test_loss +=loss_fn(pred,y).item()
            correct+=(pred.argmax(1)==y).type(torch.float).sum().item()

    test_loss /=num_batches
    correct /=size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
for t in range(epochs):
    print('epochs:',t+1)
    train_loop(train_dataloader,model,loss_fn,optimizer)
    test_loop(test_dataloader,model,loss_fn)
print('done')

epochs: 1
loss: 1.538667  [   64/60000]
loss: 1.507123  [ 6464/60000]
loss: 1.346920  [12864/60000]
loss: 1.423602  [19264/60000]
loss: 1.306327  [25664/60000]
loss: 1.310154  [32064/60000]
loss: 1.328665  [38464/60000]
loss: 1.253428  [44864/60000]
loss: 1.288324  [51264/60000]
loss: 1.196479  [57664/60000]
Test Error: 
 Accuracy: 63.1%, Avg loss: 1.218716 

epochs: 2
loss: 1.295770  [   64/60000]
loss: 1.284471  [ 6464/60000]
loss: 1.109471  [12864/60000]
loss: 1.223870  [19264/60000]
loss: 1.093393  [25664/60000]
loss: 1.122850  [32064/60000]
loss: 1.152615  [38464/60000]
loss: 1.086286  [44864/60000]
loss: 1.128263  [51264/60000]
loss: 1.051637  [57664/60000]
Test Error: 
 Accuracy: 64.4%, Avg loss: 1.068409 

epochs: 3
loss: 1.137129  [   64/60000]
loss: 1.147280  [ 6464/60000]
loss: 0.956001  [12864/60000]
loss: 1.101308  [19264/60000]
loss: 0.965585  [25664/60000]
loss: 0.998361  [32064/60000]
loss: 1.044072  [38464/60000]
loss: 0.981565  [44864/60000]
loss: 1.023653  [51264/600

In [13]:
import torchvision.models as models

In [14]:
torch.save(model.state_dict(),'model_weights.pth')

To load model weights, you need to create an instance of the same model first

In [15]:
model = NeuralNetwork()
model.load_state_dict(torch.load('model_weights.pth'))
models.eval()

<All keys matched successfully>

In [16]:
torch.save(model, 'model.pth') #能同时保存模型结构

In [17]:
model = torch.load('model.pth')